## Collecting web page data  

The purpose of this program is to scrape data from the website https://pokemondb.net/go/pokedex, 
and save it as a csv file in a format that facilitates analysis.

It will seek to answer Question 2: Is there any difference in a Pokémon’s power or skill between the two platforms? How are they different?

The following tutorial was referenced to complete this webscraping project
https://towardsdatascience.com/web-scraping-html-tables-with-python-c9baba21059

##### Inspect HTML
Before moving forward, we need to understand the structure of the website we wish to scrape.  
This can be done by clicking right-clicking the element we wish to scrape and then hitting “Inspect”.   
The Pokemon GO pokedex table is under the tag, table id="pokedex.

![title](img/table.png)

##### Import Libraries
We will need requests for getting the HTML contents of the website and lxml.html for parsing the relevant fields. 
Finally, we will store the data on a Pandas Dataframe.

In [1]:
import requests          # to get a response from a url
import lxml.html as lh   # lxml parser
import pandas as pd      # pandas dataframe

##### Scrape Table Cells
Use code below to get the Pokemon Go data of the HTML table.

In [2]:
# Assign the url to an url variable
url='https://pokemondb.net/go/pokedex'

# Create a handle, page (a response class which contains a server's response to an HTTP request), 
# to handle the contents of the website
page = requests.get(url)

In [3]:
# use the status_code method, a function defined in the class, to check the http response status code
page.status_code # the url is responsed successfully

200

HTTP response status codes indicate whether a specific HTTP request has been successfully completed. Responses are grouped in five classes:  

Informational responses (100–199),  
Successful responses (200–299),  
Redirects (300–399),  
Client errors (400–499),  
and Server errors (500–599).  

In [4]:
# use the content method, a function defined in the class, to take a quickly look at the html code
page.content[:500]

b'<!DOCTYPE html>\n<html lang="en">\n<head>\n<meta charset="utf-8">\n<title>Pok\xc3\xa9mon GO list of Pok\xc3\xa9mon | Pok\xc3\xa9mon Database</title>\n<link rel="preconnect" href="https://fonts.gstatic.com">\n<link rel="preconnect" href="https://img.pokemondb.net">\n<link rel="stylesheet" href="/static/css/pokemondb-3866aea80e.css">\n<meta name="viewport" content="width=device-width, initial-scale=1">\n<meta property="og:description" name="description" content="Complete Pok\xc3\xa9dex for all Pok\xc3\xa9mon available in Pok\xc3\xa9mon GO.">'

In [5]:
# use fromstring() method from lxml.html module to store bytes type to an Element class
doc = lh.fromstring(page.content)  # <lxml.html.HtmlElement> 

####  < table > tag:
The < table > tag defines an HTML table.  
An HTML table consists of the < table > element and one or more < tr >, < th >, and < td > elements.  
The < tr > element defines a table row,  
The < th > element defines a table header,  
and the < td > element defines a table cell.  

A more complex HTML table may also include < caption >, < col >, < colgroup >, < thead >, < tfoot >, and < tbody > elements.  

##### Example:   
< table>  
....< tr>  
........< th>Month< /th>  
........< th>Savings< /th>  
....< / tr>  
....< tr>  
........< td> January< / td>  
........< td> 100< / td>  
....< / tr>  
....< tr>  
........< td>February< /td>  
........< td> 80< /td>  
....< /tr>  
< /table>  

In [6]:
# Parse data that are stored between <tr>..</tr> of HTML by using xpath() method from lxml module
# the xpath() method will return a list
tr_elements = doc.xpath('//tr')
type(tr_elements)   # <list>

list

In [7]:
# Check each row has the same length
# Here only check the previous 12 rows
[len(T) for T in tr_elements[:12]]

[11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11]

It seems like all rows have exactly 11 columns.  
It means all the data collected on tr_elements are from the correct table.

##### Parse Table Header
Next, we need to parse the first row as the header.

In [8]:
type(tr_elements[0].text_content())

lxml.etree._ElementUnicodeResult

In [9]:
# Create empty list to store headers
col=[]
# For loop the first row as the headers
for i,t in enumerate(tr_elements[0]):
    # use text_content() method of lh module to get the row content
    name = t.text_content() 
    # print the header for each column
    print('{:2d} : {:s}'.format(i+1,name))
    # Append tuples that contain each first element (header) and an empty list (to fill the cells from the table)
    col.append((name,[]))

 1 : #
 2 : Name
 3 : Type
 4 : Attack
 5 : Defense
 6 : HP
 7 : Catch Rate
 8 : Flee Rate
 9 : Candy
10 : Fast Moves
11 : Charge Moves


In [10]:
# col[0] contains column names; col[1] is to store cells information (rows)
col

[('#', []),
 ('Name', []),
 ('Type', []),
 ('Attack', []),
 ('Defense', []),
 ('HP', []),
 ('Catch Rate', []),
 ('Flee Rate', []),
 ('Candy', []),
 ('Fast Moves', []),
 ('Charge Moves', [])]

##### Creating Pandas DataFrame  
Each header is appended to a tuple along with an empty list.

In [11]:
# Since the first row is the header, data is stored on the second row onwards
# the range of the tr_elements starts from 1
for j in range(1,len(tr_elements)):
    # T is our j'th row
    T = tr_elements[j]
    
    # Check the number of columns for each row
    # If row is not of size 11, the '//tr' data is not from our table 
    if len(T)!=11:
        break
    
    # i is the index counter of our column
    # it will be reset as 0 when a new row is iterated 
    i=0
    
    # Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        # Convert any numerical value to integers
        # The table on the web has the numeric value in integer, instead of float type
        # Try to convert the data into numeric
        # If it cannot be converted, use the except and pass keep the data as a string
            try:
                data=int(data)
            except:
                pass
        # Append the data to the empty list of the i'th column
        col[i][1].append(data) 
        # Increment i for the next column
        i+=1

In [12]:
# check the number of rows for each column
[len(cell) for colname, cell in col]

[570, 570, 570, 570, 570, 570, 570, 570, 570, 570, 570]

##### Create the DataFrame:
The dataframe contains 11 columns and 570 rows.

In [13]:
# convert a list of tuples to dictionary
Dict={colname : column for (colname , column) in col}

In [14]:
import pandas as pd 
df = pd.DataFrame(Dict)
df

,#,Name,Type,Attack,Defense,HP,Catch Rate,Flee Rate,Candy,Fast Moves,Charge Moves
0,001,Bulbasaur,Grass Poison,128,118,111,20%,10%,25,Vine WhipTackle,Seed BombSludge BombPower Whip
1,002,Ivysaur,Grass Poison,155,151,143,10%,7%,100,Vine WhipRazor Leaf,Sludge BombSolar BeamPower Whip
2,003,Venusaur,Grass Poison,190,198,189,5%,5%,—,Vine WhipRazor Leaf,Petal BlizzardSludge BombSolar Beam
3,004,Charmander,Fire,118,116,93,20%,10%,25,EmberScratch,FlamethrowerFlame ChargeFlame Burst
4,005,Charmeleon,Fire,151,158,126,10%,7%,100,EmberFire Fang,FlamethrowerFlame BurstFire Punch
...,...,...,...,...,...,...,...,...,...,...,...
565,633,Deino,Dark Dragon,141,116,93,40%,9%,25,Dragon BreathTackle,Dragon PulseBody SlamCrunch
566,634,Zweilous,Dark Dragon,176,159,135,10%,7%,100,BiteDragon Breath,Dark PulseDragon PulseBody Slam
567,635,Hydreigon,Dark Dragon,211,256,188,5%,5%,—,BiteDragon Breath,Dark PulseFlash CannonDragon Pulse
568,808,Meltan,Steel,130,118,99,30%,0%,400,Thunder Shock,Flash CannonThunderbolt


In order to do analysis on this table, there are some steps we need to do:
1. Rename the column names   
    (use 'GO' as suffix to indicate it is Pokemon Go table)  
2. Columns transformation  (Name, Type , Fast Moves, and Charge Moves)  
    - Define functions to apply on each column
        + Use name_bracket() on 'Name' column
        + Use split_2type() on 'Type' column
        + Use camel_case_split() on 'Fast Moves' column
        + Use camel_case_split() on 'Charge Moves' column
3. Replace '—' with 0  

##### Split Type column

In [15]:
def split_2type(cell_str):
    # strip the leading or trailing spaces, then split
    result = cell_str.strip().split()
    return result

In [16]:
# use apply method from pandas to apply a function to each row
# Pandas.apply allow the users to pass a function and apply it on every single value of the Pandas series.
# https://www.geeksforgeeks.org/python-pandas-apply/
df['Type'] = df['Type'].apply(split_2type)
df['Type']

0      [Grass, Poison]
1      [Grass, Poison]
2      [Grass, Poison]
3               [Fire]
4               [Fire]
            ...       
565     [Dark, Dragon]
566     [Dark, Dragon]
567     [Dark, Dragon]
568            [Steel]
569            [Steel]
Name: Type, Length: 570, dtype: object

##### Add bracket to the special name 

In [17]:
def name_bracket(name):  # ex: name='Vulpix Alolan Vulpix'
    testlist = name.split()
    # check if the first word is duplicated
    if testlist[0] in testlist[1:len(testlist)]:
        # split the charaters into a list
        namelist = [x for x in name]   # ex: ['V', 'u', 'l',  'p'....etc]
        
        for char_index in range(1, len(namelist)):
            # check which character is uppder case character except the first character
            if namelist[char_index].isupper():
            # add a leading space before the upper case character
                namelist[char_index] = ' ' + namelist[char_index] # ex: from 'A' to ' A'
        # Now we need to comobine a list of characters together and split the word by spaces
        final_list = ''.join(namelist).split(' ')  # ['Vulpix', '', 'Alolan', '', 'Vulpix']
        # replace a '(' at index 1
        final_list[1]='('
        # append a ')' to the list
        final_list.append(')') 
        # combine all the characters with parentheses
        return ' '.join(final_list)
    else:
        return name

In [18]:
# use apply method from pandas to apply a function to each row
# Pandas.apply allow the users to pass a function and apply it on every single value of the Pandas series.
# https://www.geeksforgeeks.org/python-pandas-apply/
df['Name'] = df['Name'].apply(name_bracket)
df['Name']

0       Bulbasaur
1         Ivysaur
2        Venusaur
3      Charmander
4      Charmeleon
          ...    
565         Deino
566      Zweilous
567     Hydreigon
568        Meltan
569      Melmetal
Name: Name, Length: 570, dtype: object

##### Break string bewteen lower case and upper case

In [19]:
def camel_case_split(strings): 
    'Break string bewteen lower case and upper case' 
    # split the input string to check the number of possible words
    stringlist=strings.split()
    # check if there is at least a word (it may have no word in a cell)
    if len(stringlist)>=1: 
        words = [strings[0]]
        for c in strings[1:]:
            # When the last character is lower case and the following word is upper case
            if words[-1].islower() and c.isupper():  # list[-1]: last item in the array
                # append a delimiter '-'
                words.append('-')
                # append the followeing upper case character
                words.append(c) 
            # Either the last character is upper case or the following word is lower case
            else: 
                # append the followeing lower case character
                words.append(c)
                # combine a list of words to a string
                combine=''.join(words)
            # return a list of seperated strings delimited at '-'
        return combine.split('-')
    # if a cell is empty, return itself
    else:
        return strings

In [20]:
# use apply method from pandas to apply a function to each row
# Pandas.apply allow the users to pass a function and apply it on every single value of the Pandas series.
# https://www.geeksforgeeks.org/python-pandas-apply/
df['Fast Moves']=df['Fast Moves'].apply(camel_case_split)
df['Fast Moves']

0          [Vine Whip, Tackle]
1      [Vine Whip, Razor Leaf]
2      [Vine Whip, Razor Leaf]
3             [Ember, Scratch]
4           [Ember, Fire Fang]
                ...           
565    [Dragon Breath, Tackle]
566      [Bite, Dragon Breath]
567      [Bite, Dragon Breath]
568            [Thunder Shock]
569            [Thunder Shock]
Name: Fast Moves, Length: 570, dtype: object

In [21]:
# use apply method from pandas to apply a function to each row
# Pandas.apply allow the users to pass a function and apply it on every single value of the Pandas series.
# https://www.geeksforgeeks.org/python-pandas-apply/
df['Charge Moves']=df['Charge Moves'].apply(camel_case_split)
df['Charge Moves']

0                   [Seed Bomb, Sludge Bomb, Power Whip]
1                  [Sludge Bomb, Solar Beam, Power Whip]
2              [Petal Blizzard, Sludge Bomb, Solar Beam]
3              [Flamethrower, Flame Charge, Flame Burst]
4                [Flamethrower, Flame Burst, Fire Punch]
                             ...                        
565                    [Dragon Pulse, Body Slam, Crunch]
566                [Dark Pulse, Dragon Pulse, Body Slam]
567             [Dark Pulse, Flash Cannon, Dragon Pulse]
568                          [Flash Cannon, Thunderbolt]
569    [Hyper Beam, Flash Cannon, Rock Slide, Thunder...
Name: Charge Moves, Length: 570, dtype: object

In [22]:
# Replace '—' to 0 on 'Candy' column
df['Candy']=['0' if x=='—' else x for x in df['Candy']]  

In [23]:
# rename column names
df.rename(columns={'Catch Rate':'Catch_Rate',
                   'Flee Rate':'Flee_Rate',
                   'Fast Moves':'Fast_Moves',
                   'Charge Moves':'Charge_Moves',}, inplace=True)

##### Version 1: raw data with list in a cell

In [24]:
# output a list in cells version of csv
df.to_csv('pokemonGo_list.csv', encoding='utf-8')

##### Version 2:  Each cell contains only one value
Here we are going to split column of lists into multiple columns

In [25]:
# Name column does not contain duplicate value
df['Name'].nunique()

570

In [26]:
Type =pd.DataFrame(df['Type'].values.tolist(), columns=['Type1','Type2'])

In [27]:
# split 'Fast_Moves' column
maxlist=[]
for x in df['Fast_Moves']:
    a=len(x)
    maxlist.append(a)
Fast_Moves=pd.DataFrame(df['Fast_Moves'].values.tolist(), columns=[f'Fast_Moves_{i+1}' for i in range(max(maxlist))])

In [28]:
# split 'Charge_Moves' column
maxlist=[]
for x in df['Charge_Moves']:
    a=len(x)
    maxlist.append(a)
Charge_Moves=pd.DataFrame(df['Charge_Moves'].values.tolist(), columns=[f'Charge_Moves_{i+1}' for i in range(max(maxlist))])

In [29]:
# https://songhuiming.github.io/pages/2017/04/02/jupyter-and-pandas-display/
pd.set_option('display.max_columns', 500)

# https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html
df2=pd.concat([df[['#','Name']], Type, df[['Attack','Defense','HP','Catch_Rate','Flee_Rate','Candy']], 
           Fast_Moves,Charge_Moves], axis=1)

In [30]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 570 entries, 0 to 569
Data columns (total 49 columns):
#                  570 non-null object
Name               570 non-null object
Type1              570 non-null object
Type2              272 non-null object
Attack             570 non-null int64
Defense            570 non-null int64
HP                 570 non-null int64
Catch_Rate         570 non-null object
Flee_Rate          570 non-null object
Candy              570 non-null object
Fast_Moves_1       568 non-null object
Fast_Moves_2       553 non-null object
Fast_Moves_3       56 non-null object
Fast_Moves_4       7 non-null object
Fast_Moves_5       2 non-null object
Fast_Moves_6       1 non-null object
Fast_Moves_7       1 non-null object
Fast_Moves_8       1 non-null object
Fast_Moves_9       1 non-null object
Fast_Moves_10      1 non-null object
Fast_Moves_11      1 non-null object
Fast_Moves_12      1 non-null object
Fast_Moves_13      1 non-null object
Fast_Moves_14      1 n

In [31]:
# output a list in cells version of csv
df2.to_csv('pokemonGo_wide.csv', encoding='utf-8')